In [ ]:
from sys import path
from os.path import dirname, abspath
path.append(dirname(dirname(dirname(abspath("__file__")))))

In [ ]:
# for NER model
from transformers import AutoModel
from torchcrf import CRF
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from scripts.utils_bert import get_encoded_input as get_encoded_input_bert
from scripts.utils import get_encoded_input as get_encoded_input_seq2seq
from scripts.metrics import f1score
import pytorch_lightning as pl
from multiprocessing import cpu_count
from platform import system
from os import environ
from itertools import chain
from sklearn.metrics import f1_score

# set the environment to run tokenizers in parallel
environ["TOKENIZERS_PARALLELISM"] = "false"
pl.seed_everything(seed=42)

### NER Model (Definition)
#### NER model is only used in inference mode by loading the saved_weights
#### Hence, all training elements of the model can be removed

In [ ]:
# some important hyperparameters
BATCH_SIZE = 128
MAX_LEN = None
N_JOBS = cpu_count() if system() != "Windows" else 0

BERT_TYPE = "roberta-base"
CELL_TYPE = "lstm"
MODEL_NAME = f"seq2seq-{CELL_TYPE}-ner-with-pos-tags" #remove uni for bidirectional
TAG2IDX = {'B': 0, 'I': 1, 'O': 2, 'E': 3, 'S': 4, '<': 5, ">":6, "$": 7}

POS_TAGS2IDX = {'PAD_AUX': 0, 'ADJ': 1, 'ADP': 2, 'ADV': 3, 'AUX': 4, 
                'CONJ': 5, 'CCONJ': 6, 'DET': 7, 'INTJ': 8, 'NOUN': 9, 
                'NUM': 10, 'PART': 11, 'PRON': 12, 'PROPN': 13, 'PUNCT': 14, 
                'SCONJ': 15, 'SYM': 16, 'VERB': 17, 'X': 18, 'SPACE': 19}

### Seq2Seq with POS-tags

In [ ]:
class SEQ2SEQ_POS_TAGS(pl.LightningModule):
    def __init__(self, 
                 input_dim1, 
                 input_dim2,
                 cell_type="lstm",
                 embed_dim1=128, 
                 embed_dim2=32,
                 dropout=0.5, 
                 cell_dim1=128,
                 cell_dim2=128,
                 bidirectional=True, 
                 num_layers1=3,
                 num_layers2=1,
                 num_tags=len(TAG2IDX),
                 test_dataset=None):

        super().__init__()
        self.embedding1 = nn.Embedding(num_embeddings=input_dim1,
                                       embedding_dim=embed_dim1,
                                       padding_idx=TAG2IDX['$'])

        self.embedding2 = nn.Embedding(num_embeddings=input_dim2,
                                       embedding_dim=embed_dim2,
                                       padding_idx=POS_TAGS2IDX['PAD_AUX'])
        
        c = (2 if bidirectional else 1)
        if cell_type == "lstm":
            self.cell1 = nn.LSTM(input_size=embed_dim1, 
                                hidden_size=cell_dim1, 
                                dropout=dropout,
                                batch_first=True,
                                num_layers=num_layers1, 
                                bidirectional=bidirectional)

            self.cell2 = nn.LSTM(input_size=embed_dim2, 
                                hidden_size=cell_dim2, 
                                dropout=dropout,
                                batch_first=True,
                                num_layers=num_layers2, 
                                bidirectional=bidirectional)
        else:
            self.cell1 = nn.GRU(input_size=embed_dim1, 
                                hidden_size=cell_dim1, 
                                dropout=dropout,
                                batch_first=True,
                                num_layers=num_layers1, 
                                bidirectional=bidirectional)

            self.cell2 = nn.GRU(input_size=embed_dim2, 
                                hidden_size=cell_dim2, 
                                dropout=dropout,
                                batch_first=True,
                                num_layers=num_layers2, 
                                bidirectional=bidirectional)

        self.fc = nn.Linear(c*cell_dim1, num_tags)
        self.crf = CRF(num_tags=num_tags, batch_first=True)
        self.dropout = nn.Dropout(p=dropout)
        ## Hyperparameters ##
        self.batch_size = BATCH_SIZE
        ## Datasets ##
        self.test_dataset = test_dataset


    def test_dataloader(self):
        return DataLoader(self.test_dataset, 
                          batch_size=self.batch_size,
                          num_workers=N_JOBS,
                          shuffle=False,
                          drop_last=False)
    

    def forward(self, input_ids1, input_ids2):
        out1, _ = self.cell1(self.embedding1(input_ids1))
        out2, _ = self.cell2(self.embedding2(input_ids2))
        out = out1 + out2
        out = F.relu(out)
        out = self.dropout(out)
        out = self.fc(out)
        return out


    def _shared_evaluation_step(self, batch, batch_idx):
        ids1, ids2, masks, lbls = batch
        emissions = self(ids1, ids2)
        loss = -self.crf(emissions, lbls, mask=masks)
        pred = self.crf.decode(emissions, mask=masks)
        r, p, f1 = f1score(lbls, pred)
        return loss, r, p, f1
     
    
    def test_step(self, batch, batch_idx):
        loss, r, p, f1 = self._shared_evaluation_step(batch, batch_idx)
        self.log("test_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_recall", r, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_precision", p, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_f1score", f1, on_step=False, on_epoch=True, prog_bar=True)

### RoBERTa-Classification

In [ ]:
class BERT_SUG(pl.LightningModule):
    def __init__(self, 
                 dropout=0.5,
                 bert_type=BERT_TYPE,
                 test_dataset=None):

        super().__init__()
        self.bert = AutoModel.from_pretrained(bert_type)
        self.fc = nn.Linear(768, 1)
        self.dropout = nn.Dropout(p=dropout)
        self.layer_norm = nn.LayerNorm(768)
        self.loss_fn = nn.BCEWithLogitsLoss()
        ## Hyperparameters ##
        self.batch_size = BATCH_SIZE
        ## Datasets ##
        self.test_dataset = test_dataset


    def test_dataloader(self):
        return DataLoader(self.test_dataset, 
                          batch_size=self.batch_size,
                          num_workers=N_JOBS,
                          shuffle=False,
                          drop_last=False)

    def predict_dataloader(self):
        return DataLoader(self.test_dataset, 
                          batch_size=self.batch_size,
                          num_workers=N_JOBS,
                          shuffle=False,
                          drop_last=False)
    
    
    def _f1score(self, logits, lbls):
        lbls = torch.flatten(lbls)
        preds = torch.flatten(torch.round(torch.sigmoid(logits)))
        return f1_score(lbls.tolist(), preds.tolist(), zero_division=0)
        

    def forward(self, input_ids, attention_masks):
        out = self.bert(input_ids, attention_masks).pooler_output
        out = self.layer_norm(out)
        out = self.dropout(out)
        out = self.fc(out)
        return out
    
    
    def _shared_evaluation_step(self, batch, batch_idx):
        ids, masks, lbls = batch
        logits = self(ids, masks)
        loss = self.loss_fn(logits, lbls.float())
        f1 = self._f1score(logits, lbls)
        return loss, f1


    def test_step(self, batch, batch_idx):
        loss, f1 = self._shared_evaluation_step(batch, batch_idx)
        self.log("test_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_f1score", f1, on_step=False, on_epoch=True, prog_bar=True)


    def predict_step(self, batch, batch_idx):
        ids, masks, _ = batch
        logits = self(ids, masks)
        preds = torch.flatten(torch.round(torch.sigmoid(logits)))
        return preds.tolist()


### Use RoBERTa-Classification Model for predicting Suggestions

In [ ]:
# similarly read the test data and create the dataset
encoded_input, targets = get_encoded_input_bert("../../../data/test_290818.txt", 
                                                tag2idx=TAG2IDX, 
                                                tokenizer_name=BERT_TYPE,
                                                return_classification_targets=True)

test_dataset = TensorDataset(torch.LongTensor(encoded_input["input_ids"]),
                             torch.BoolTensor(encoded_input["attention_mask"]),
                             torch.LongTensor(targets).unsqueeze(1))

In [ ]:
model = BERT_SUG(bert_type=BERT_TYPE,
                 test_dataset=test_dataset)

trainer = pl.Trainer(accelerator="gpu",
                     precision=16,
                     log_every_n_steps=1)

In [ ]:
model.load_state_dict(torch.load(f"../../../src_sug/saved_weights/{BERT_TYPE}-sug-on-NER-dataset.ckpt")["state_dict"])
trainer.test(model)

In [ ]:
preds = list(chain(*trainer.predict(model)))

### Use the sentences which are predicted to have a Suggestion as input to NER Model

In [ ]:

sug_idx = torch.LongTensor([i for (i, y) in enumerate(preds) if y == 1.0])

In [ ]:
with open("../../../data/full_vocab_ner.txt", mode="r", encoding="utf-8") as f:
    vocab = [s.strip() for s in f.readlines()]
    VOCAB2IDX = {v:k for (k, v) in enumerate(vocab)}

In [ ]:
encoded_input, pos_tags, masks, extended_labels = get_encoded_input_seq2seq("../../../data/test_290818.txt", 
                                                                            tag2idx=TAG2IDX,
                                                                            vocab2idx=VOCAB2IDX,
                                                                            pos_tags2idx=POS_TAGS2IDX,
                                                                            return_pos_tags=True,
                                                                            maxlen=MAX_LEN)

encoded_input = torch.index_select(torch.LongTensor(encoded_input), dim=0, index=sug_idx)
pos_tags = torch.index_select(torch.LongTensor(pos_tags), dim=0, index=sug_idx)
masks = torch.index_select(torch.BoolTensor(masks), dim=0, index=sug_idx)
extended_labels = torch.index_select(torch.LongTensor(extended_labels), dim=0, index=sug_idx)

test_dataset = TensorDataset(encoded_input, pos_tags, masks, extended_labels)

In [ ]:
model = SEQ2SEQ_POS_TAGS(input_dim1=len(VOCAB2IDX),
                         input_dim2=len(POS_TAGS2IDX),
                         cell_type=CELL_TYPE,
                         bidirectional=True,
                         test_dataset=test_dataset)

trainer = pl.Trainer(accelerator="gpu",
                     precision=16,
                     log_every_n_steps=1)

In [ ]:
model.load_state_dict(torch.load(f"../../saved_weights/{MODEL_NAME}.ckpt")["state_dict"])
trainer.test(model)